In [1]:
import sys
realmin = sys.float_info.min

import tensorflow as tf
import time
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import time

from scipy.optimize import minimize
from numba import jit, njit

2024-10-19 16:47:35.211748: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 16:47:35.217020: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-19 16:47:35.234668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 16:47:35.264348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 16:47:35.273207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 16:47:35.292947: I tensorflow/core/platform/cpu_feature_gu

### Adam Optimization Algorithm

#### Adam Tensorflow

In [2]:
def adam_tensorflow(fun, x0, tol):
    start_time = time.time()

    x = tf.Variable(x0) # Define the variable x (initial guess) as a tensor

    tolerance = tol # Set your desired tolerance value

    learning_rate = 0.1
    # The learning rate. Defaults to `0.001`.

    beta_1 = 0.9
    # A float value or a constant float tensor, or a callable that
    # takes no arguments and returns the actual value to use. The
    # exponential decay rate for the 1st moment estimates. Defaults to `0.9`.

    beta_2 = 0.999
    # A float value or a constant float tensor, or a callable
    # that takes no arguments and returns the actual value to use. The
    # exponential decay rate for the 2nd moment estimates. Defaults to `0.999`.

    epsilon = 1e-8
    # A small constant for numerical stability. This epsilon is "epsilon hat" in
    # the Kingma and Ba paper (in the formula just before Section 2.1),
    # not the epsilon in Algorithm 1 of the paper. Defaults to `1e-7`.

    iteration_limit = 5000

    optimizer = tf.optimizers.Adam(
        learning_rate, beta_1, beta_2, epsilon
    )

    # Initialize a variable to store the previous value of the objective function
    prev_loss = float('inf')

    iter = 0
    # Perform optimization with iteration limit and function tolerance
    while True:
        iter += 1

        with tf.GradientTape() as tape:
            loss = fun(x) # Define the objective function

        # Compute the gradient of the loss with respect to x
        grads = tape.gradient(loss, [x])

        # Apply the gradients to update x
        optimizer.apply_gradients(zip(grads, [x]))

        # Check if the difference in the loss is less than the tolerance
        loss_change = abs(prev_loss - loss.numpy())
        if loss_change < tolerance:
            break # Stop the optimization when tolerance is met

        # Update the previous loss
        prev_loss = loss.numpy()

        if iter >= iteration_limit:
            break # Stop the optimization when tolerance is met

    end_time = time.time()

    class solution:
        def __init__(self):
            self.x = x
            self.func = fun(x)
            self.iters = iter
            self.time = end_time - start_time

    return solution()

In [3]:
def objective(x):
    return (x[0] - 7)**2 + (x[1] - 2)**2

In [4]:
result = adam_tensorflow(
    fun = objective,
    x0 = [0.0, 0.0],
    tol = 0.01
)

In [5]:
result.x

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([6.704236 , 1.9979095], dtype=float32)>

#### Adam Njit

In [6]:
@njit
def objective_function(x):
    return (x[0] - 7)**2 + (x[1] - 2)**2

In [7]:
@njit
def adam_gradient(x):
    """Numerical gradient approximation using central differences."""

    eps = 1e-8
    grad = np.zeros_like(x)

    for i in range(len(x)):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += eps
        x_minus[i] -= eps
        grad[i] = (objective_function(x_plus) - objective_function(x_minus)) / (2 * eps)

    return grad

In [8]:
@njit
def adam_solver_numerical(x0, tol):
    x = x0.copy() # Initial guess

    tolerance = tol # Set your desired tolerance value

    learning_rate = 0.1
    # The learning rate. Defaults to `0.001`.

    beta_1 = 0.9
    # A float value or a constant float tensor, or a callable that
    # takes no arguments and returns the actual value to use. The
    # exponential decay rate for the 1st moment estimates. Defaults to `0.9`.

    beta_2 = 0.999
    # A float value or a constant float tensor, or a callable
    # that takes no arguments and returns the actual value to use. The
    # exponential decay rate for the 2nd moment estimates. Defaults to `0.999`.

    epsilon = 1e-8
    # A small constant for numerical stability. This epsilon is "epsilon hat" in
    # the Kingma and Ba paper (in the formula just before Section 2.1),
    # not the epsilon in Algorithm 1 of the paper. Defaults to `1e-7`.

    iteration_limit = 5000

    # Moment estimates
    m = np.zeros_like(x) # Initialize 1st moment vector
    v = np.zeros_like(x) # Initialize 2nd moment vector
    iter = 0 # Iteration counter

    # Initialize a variable to store the previous value of the objective function
    prev_loss = objective_function(x)

    # Perform optimization
    while True:
        iter += 1
        # Compute the gradient of the loss with respect to x
        grads = adam_gradient(x)

        # Update biased first moment estimate
        m = beta_1 * m + (1 - beta_1) * grads
        # Update biased second raw moment estimate
        v = beta_2 * v + (1 - beta_2) * (grads ** 2)

        # Compute bias-corrected first moment estimate
        m_hat = m / (1 - beta_1 ** iter)
        # Compute bias-corrected second raw moment estimate
        v_hat = v / (1 - beta_2 ** iter)

        # Update the variable x
        x -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

        # Compute current loss
        loss = objective_function(x)

        loss_change = abs(prev_loss - loss)
        if loss_change < tolerance or iter >= iteration_limit:
            break # Stop the optimization when tolerance is met or max iterations reached

        # Update the previous loss
        prev_loss = loss

    return x, iter

In [9]:
x0 = np.array([0.0, 0.0])  # Initial guess
tol = 1e-6  # Desired tolerance

start_time = time.time()
x, iter = adam_solver_numerical(x0, tol)
end_time = time.time()

# Define the solution as a class with final results
class solution:
    def __init__(self):
        self.x = x
        self.func = objective_function(x)
        self.iters = iter
        self.time = end_time - start_time

result = solution()

print(f"Optimal x: {result.x}")
print(f"Function value: {result.func}")
print(f"Iterations: {result.iters}")
print(f"Time taken: {round(result.time,4)} seconds")

Optimal x: [6.9996924  1.99966012]
Function value: 2.101366960765992e-07
Iterations: 165
Time taken: 1.5965 seconds
